In [1]:
library(EnhancedVolcano)
library(Seurat)
library(ggplot2)
library(DoubletFinder)
library(dplyr)
library(stringr)
library(RColorBrewer)

Loading required package: ggplot2

Loading required package: ggrepel

Loading required package: SeuratObject

Loading required package: sp


Attaching package: ‘SeuratObject’


The following objects are masked from ‘package:base’:

    %||%, intersect, t




Error in .make_numeric_version(x, strict, .standard_regexps()$valid_numeric_version) : 
  invalid non-character version specification 'x' (type: double)



Attaching package: ‘Seurat’


The following object is masked from ‘package:base’:

    %||%



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [ ]:
Young.data <- read.csv("YAP5SA_DropSeq.raw.data.csv", sep=",")
Youngmeta.data <- read.csv("YAP5SA_DropSeq.meta.data.csv", sep=",")
CM_P1_ShamD1 <- Read10X("OlsenCMOnly/GSM3747857_P1ShamD1/")
CM_P1_ShamD3 <- Read10X("OlsenCMOnly/GSM3747859_P1ShamD3/")
CM_P8_ShamD1 <- Read10X("OlsenCMOnly/GSM3747861_P8ShamD1/")
CM_P8_ShamD3 <- Read10X("OlsenCMOnly/GSM3747863_P8ShamD3/")

In [ ]:
Young <- CreateSeuratObject(Young.data, meta.data = Youngmeta.data, min.cells = 10, min.features = 500) 
Young  <- PercentageFeatureSet(Young, pattern="mt-", col.name="percent.mt")
Data <- ls(pattern = "_P")
DataList <- mget(c(ls(pattern = "_P")))

for (i in Data)
{
  x <- CreateSeuratObject(DataList[[i]], min.cells = 10, min.features = 200)
  x <- PercentageFeatureSet(x, pattern="mt", col.name="percent.mt")
  x$orig.ident <- i
  assign(i,x)
  rm(x)
}


In [ ]:
Data <- c(ls(pattern = "_P"), ls(pattern = "Young"))
DataList <- mget(c(ls(pattern = "_P"), ls(pattern = "Young")))

for (i in Data)
{
  x <- SCTransform(DataList[[i]], vars.to.regress = c("percent.mt","nFeature_RNA"), verbose = FALSE)
  x$orig.ident <- i
  assign(i,x)
  print(i)
  rm(x)
}

In [ ]:
CM_P1_ShamD1$Experiment <- "P1ShamD1"
CM_P1_ShamD3$Experiment <- "P1ShamD3"
CM_P8_ShamD1$Experiment <- "P8ShamD1"
CM_P8_ShamD3$Experiment <- "P8ShamD3"

In [ ]:
Data <- c(ls(pattern = "_P"), ls(pattern = "Young"))
DataList <- mget(c(ls(pattern = "_P"), ls(pattern = "Young")))

In [ ]:
features <- SelectIntegrationFeatures(object.list = DataList, nfeatures = 15000)
List <- PrepSCTIntegration(object.list=DataList,anchor.features=features, verbose=FALSE)

In [ ]:
Combined.anchors <- FindIntegrationAnchors(List, dims = 1:40, normalization.method = "SCT",anchor.features = features, verbose=FALSE)
Combined.Integrated <- IntegrateData(Combined.anchors, normalization.method = "SCT", dims=1:40)

In [ ]:
Combined.Integrated <- RunPCA(Combined.Integrated, npcs=40, verbose=FALSE)

Combined.Integrated <- RunUMAP(Combined.Integrated, reduction="pca", dims=1:20)
Combined.Integrated <- FindNeighbors(Combined.Integrated, reduction="pca", dims=1:20)
Combined.Integrated <- FindClusters(Combined.Integrated, resolution=.1)

In [ ]:
DimPlot(Combined.Integrated, 
        reduction="umap", 
        label=TRUE, 
        group.by="integrated_snn_res.0.1", 
        label.size = 3,
        cols=brewer.pal(3,"Set2"))

In [ ]:
CM <- subset(Combined.Integrated, idents=c("0","3"))

In [ ]:
Idents(CM) <- CM$Experiment
CMControl <- subset(CM, idents=c("Ctrl","P1ShamD1"))

In [ ]:
DefaultAssay(CMControl) <- 'integrated'
CMControl <- RunPCA(CMControl, npcs=40, verbose=FALSE)
CMControl <- RunUMAP(CMControl, reduction="pca", dims=1:5, min.dist = 1)
CMControl <- FindNeighbors(CMControl, reduction="pca", dims=1:5)
DefaultAssay(CMControl) <- 'integrated'
CMControl <- FindClusters(CMControl, resolution=.1)

In [ ]:
Idents(CMControl) <- CMControl$Experiment
DefaultAssay(CMControl) <- "SCT"
CMControlDEG <- FindMarkers(CMControl, ident.1="Ctrl", ident.2="P1ShamD1")
CMControlDEG <- CMControlDEG[!grepl("mt-", rownames(CMControlDEG)),]

In [ ]:
CMControlDEGDown <- subset(CMControlDEG, subset=avg_log2FC < 0)
CMControlDEGUp <- subset(CMControlDEG, subset=avg_log2FC > 0)
CMControlDEGDown <- CMControlDEGDown[order(CMControlDEGDown$avg_log2FC),]
CMControlDEGUp <- CMControlDEGUp[order(CMControlDEGUp$avg_log2FC),]

In [ ]:
DefaultAssay(CMControl) <- "SCT"
Idents(CMYap5SA) <- CMYap5SA$Experiment
CMYap5SA_Avg_Exp <- AverageExpression(CMYap5SA, assay='SCT',slot='data',group.by="Experiment")

In [ ]:
CMYap5SA_Avg_Exp_Down <- data.frame(CMYap5SA_Avg_Exp$SCT[rownames(CMControlDEGDown),])
CMYap5SA_Avg_Exp_Up <- data.frame(CMYap5SA_Avg_Exp$SCT[rownames(CMControlDEGUp),])

In [ ]:
CMYap5SA_Avg_Exp_Down <- CMYap5SA_Avg_Exp_Down[order(CMYap5SA_Avg_Exp_Down$YAP5SA,decreasing=TRUE),]
CMYap5SA_Avg_Exp_Up <- CMYap5SA_Avg_Exp_Up[order(CMYap5SA_Avg_Exp_Up$YAP5SA,decreasing=TRUE),]

In [ ]:
CMYap5SA_Avg_Exp_UpTop30 <- head(CMYap5SA_Avg_Exp_Up,n = 30)
DotPlot(CMYap5SA, features = rownames(CMYap5SA_Avg_Exp_UpTop30), cols = c("lightgrey","red"),
        group.by="Experiment") + RotatedAxis() + theme(legend.position="bottom")

In [ ]:
CMYap5SA_Avg_Exp_DownTop30 <- head(CMYap5SA_Avg_Exp_Down,n = 30)
DotPlot(CMYap5SA, features = rownames(CMYap5SA_Avg_Exp_DownTop30),, cols = c("lightgrey","blue"), 
        group.by="Experiment") + RotatedAxis() + theme(legend.position="bottom")